In [16]:
import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id='8c257d6ff1fb47f7a19315e89128efef', client_secret='bfb4d4e3262241eca30015f07a07195c')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [5]:
artist_data = []
artist_uri = 'spotify:artist:06HL4z0CvFAxyc27GXpf02'

In [6]:
results = sp.artist_albums(artist_uri, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

names = []
album_ids = []
for album in albums:
    name = album['name'].lower()
    if name not in names:
        names.append(name)
        album_ids.append(album['id'])

In [7]:
for album_id in album_ids:
    results = sp.album_tracks(album_id)
    album_info = sp.album(album_id)
    tracks = results['items']
    #Results from spotify API come back in pages
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

    for track in tracks:
        track_row = {}
        track_row['spotify_id'] = track['id']
        track_row['spotify_uri'] = track['uri']
        track_row['album'] = album_info['name']
        track_row['name'] = track['name']
        track_row['release_date'] = album_info['release_date']
        track_row['track_number'] = track['track_number']
        popularity = sp.track(track['id'])['popularity']
        track_row['popularity'] = popularity
        features = sp.audio_features(track['id'])[0]
        track_row['danceability'] = features['danceability']
        track_row['energy'] = features['energy']
        track_row['key'] = features['key']
        track_row['loudness'] = features['loudness']
        track_row['mode'] = features['mode']
        track_row['speechiness'] = features['speechiness']
        track_row['acousticness'] = features['acousticness']
        track_row['instrumentalness'] = features['instrumentalness']
        track_row['liveness'] = features['liveness']
        track_row['valence'] = features['valence']
        track_row['tempo'] = features['tempo']
        track_row['duration_ms'] = features['duration_ms']
        track_row['time_signature'] = features['time_signature']
        artist_data.append(track_row)

In [11]:
cols = ['spotify_id', 'spotify_uri', 'album', 'name', 'popularity', 'release_date', 'track_number', 'danceability','energy','key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 
'tempo', 'duration_ms', 'time_signature']
artist_dataframe = pd.DataFrame(artist_data, columns = cols)  
print(artist_dataframe)

                 spotify_id                           spotify_uri  \
0    6dODwocEuGzHAavXqTbwHv  spotify:track:6dODwocEuGzHAavXqTbwHv   
1    4PdLaGZubp4lghChqp8erB  spotify:track:4PdLaGZubp4lghChqp8erB   
2    7uGYWMwRy24dm7RUDDhUlD  spotify:track:7uGYWMwRy24dm7RUDDhUlD   
3    1kbEbBdEgQdQeLXCJh28pJ  spotify:track:1kbEbBdEgQdQeLXCJh28pJ   
4    7wAkQFShJ27V8362MqevQr  spotify:track:7wAkQFShJ27V8362MqevQr   
..                      ...                                   ...   
572  15DeqWWQB4dcEWzJg15VrN  spotify:track:15DeqWWQB4dcEWzJg15VrN   
573  0JIdBrXGSJXS72zjF9ss9u  spotify:track:0JIdBrXGSJXS72zjF9ss9u   
574  5OOd01o2YS1QFwdpVLds3r  spotify:track:5OOd01o2YS1QFwdpVLds3r   
575  1spLfUJxtyVyiKKTegQ2r4  spotify:track:1spLfUJxtyVyiKKTegQ2r4   
576  4pJi1rVt9GNegU9kywjg4z  spotify:track:4pJi1rVt9GNegU9kywjg4z   

                                            album  \
0    THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY   
1    THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY   
2    THE TOR

In [12]:
columns_to_drop = ['spotify_id', 'spotify_uri', 'album']
df = artist_dataframe.drop(columns=columns_to_drop)

In [13]:
df.to_csv('output.csv', index=False)